In [1]:
pip install requests pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import pandas as pd

In [7]:
wallet_df = pd.read_excel("/Users/my475/Downloads/Wallet id.xlsx")  # Replace with your file name
wallets = wallet_df['wallet_id'].tolist()      # Column should be named 'wallet_id'

In [9]:
GRAPH_API_URL = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"

In [11]:
def run_query(query):
    response = requests.post(GRAPH_API_URL, json={'query': query})
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Query failed. Status code: {response.status_code}")
        print(response.text)
        return None


In [13]:
# Function to fetch borrow, repay, liquidation events for a wallet
def fetch_wallet_activity(wallet):
    query = f"""
    {{
      account(id: "{wallet.lower()}") {{
        id
        tokens {{
          symbol
          borrowBalanceUnderlying
          supplyBalanceUnderlying
        }}
        lifetimeBorrowInterestAccumulated
        lifetimeSupplyInterestAccumulated
        countLiquidated
        countLiquidator
      }}
    }}
    """
    result = run_query(query)
    if result and result.get('data', {}).get('account'):
        return result['data']['account']
    else:
        return None

In [15]:
wallet_data = []

for idx, wallet in enumerate(wallets):
    print(f"Fetching data for wallet {idx+1}/{len(wallets)}: {wallet}")
    data = fetch_wallet_activity(wallet)
    if data:
        wallet_data.append({
            'wallet': data['id'],
            'borrow_balance': sum(float(t['borrowBalanceUnderlying']) for t in data['tokens']),
            'supply_balance': sum(float(t['supplyBalanceUnderlying']) for t in data['tokens']),
            'lifetime_borrow_interest': float(data['lifetimeBorrowInterestAccumulated']),
            'lifetime_supply_interest': float(data['lifetimeSupplyInterestAccumulated']),
            'liquidations_count': int(data['countLiquidated'])
        })
    else:
        wallet_data.append({
            'wallet': wallet,
            'borrow_balance': 0.0,
            'supply_balance': 0.0,
            'lifetime_borrow_interest': 0.0,
            'lifetime_supply_interest': 0.0,
            'liquidations_count': 0
        })

Fetching data for wallet 1/103: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Fetching data for wallet 2/103: 0x06b51c6882b27cb05e712185531c1f74996dd988
Fetching data for wallet 3/103: 0x0795732aacc448030ef374374eaae57d2965c16c
Fetching data for wallet 4/103: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Fetching data for wallet 5/103: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Fetching data for wallet 6/103: 0x104ae61d8d487ad689969a17807ddc338b445416
Fetching data for wallet 7/103: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
Fetching data for wallet 8/103: 0x124853fecb522c57d9bd5c21231058696ca6d596
Fetching data for wallet 9/103: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
Fetching data for wallet 10/103: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
Fetching data for wallet 11/103: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
Fetching data for wallet 12/103: 0x19df3e87f73c4aaf4809295561465b993e102668
Fetching data for wallet 13/103: 0x1ab2ccad4fc97c9968ea87d4435326715be32872
Fetching data for wal

In [17]:
# STEP 4: Create DataFrame & Save

df = pd.DataFrame(wallet_data)
print(df.head())

# Save fetched data for backup
df.to_csv("compound_wallets_data.csv", index=False)
print("✅ Data saved to compound_wallets_data.csv")

                                       wallet  borrow_balance  supply_balance  \
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3             0.0             0.0   
1  0x06b51c6882b27cb05e712185531c1f74996dd988             0.0             0.0   
2  0x0795732aacc448030ef374374eaae57d2965c16c             0.0             0.0   
3  0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9             0.0             0.0   
4  0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae             0.0             0.0   

   lifetime_borrow_interest  lifetime_supply_interest  liquidations_count  
0                       0.0                       0.0                   0  
1                       0.0                       0.0                   0  
2                       0.0                       0.0                   0  
3                       0.0                       0.0                   0  
4                       0.0                       0.0                   0  
✅ Data saved to compound_wallets_data.csv


In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load wallet IDs from your Excel file
wallet_df = pd.read_excel("/Users/my475/Downloads/Wallet id.xlsx")  # Make sure your file is named correctly
wallets = wallet_df['wallet_id'].tolist()

# Simulate transaction behavior features for each wallet
np.random.seed(42)  # For reproducibility

simulated_data = []
for wallet in wallets:
    borrow_balance = np.random.uniform(0, 10000)
    supply_balance = np.random.uniform(0, 15000)
    lifetime_borrow_interest = np.random.uniform(0, 2000)
    lifetime_supply_interest = np.random.uniform(0, 2000)
    liquidations_count = np.random.poisson(0.2)  # Most will be 0, some 1

    simulated_data.append({
        'wallet': wallet,
        'borrow_balance': borrow_balance,
        'supply_balance': supply_balance,
        'lifetime_borrow_interest': lifetime_borrow_interest,
        'lifetime_supply_interest': lifetime_supply_interest,
        'liquidations_count': liquidations_count
    })

# Convert to DataFrame
df = pd.DataFrame(simulated_data)

# Derived Features
df['deposit_borrow_ratio'] = df['supply_balance'] / (df['borrow_balance'] + 1e-6)
df['repayment_efficiency'] = df['lifetime_supply_interest'] / (df['lifetime_borrow_interest'] + 1e-6)

# Fill NAs with zeros
df = df.fillna(0)

# Scoring Formula
df['raw_score'] = (
    (df['deposit_borrow_ratio'] * 300) +
    (df['repayment_efficiency'] * 400) -
    (df['liquidations_count'] * 200) -
    (df['borrow_balance'] * 0.05)
)

# Normalize raw score to 0-1000 scale
scaler = MinMaxScaler(feature_range=(0, 1000))
df['score'] = scaler.fit_transform(df[['raw_score']])

# Final CSV Output
final_df = df[['wallet', 'score']]
final_df.to_csv("wallet_risk_scores.csv", index=False)

print("✅ Simulated Wallet Risk Scores saved to wallet_risk_scores.csv")


✅ Simulated Wallet Risk Scores saved to wallet_risk_scores.csv
